In [1]:
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)
import psutil

import h5py
import gc

2024-04-03 11:26:16.359601: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.12.0
2.12.0


In [2]:
def print_memory_usage():
    memory = psutil.virtual_memory()
    print(f"Total memory: {memory.total / (1024**3):.2f} GB")
    print(f"Available memory: {memory.available / (1024**3):.2f} GB")
    print(f"Used memory: {memory.used / (1024**3):.2f} GB")
    print(f"Memory usage percentage: {memory.percent}%")

def print_cpu_usage():
    cpu_percent = psutil.cpu_percent(interval=1)
    print(f"CPU Usage: {cpu_percent}%")

In [3]:
def load_combine_shuffle_data_optimized_hdf5():
    with h5py.File('./fl32_data_v3.hdf5', 'r') as h5f:
        combined_input = None
        combined_target = None

        for data_type in ['sig', 'bkg']:
            # Construct dataset names
            input_dataset_name = f'{data_type}_input'
            target_dataset_name = f'{data_type}_target'

            # Check if the dataset exists and load data sequentially
            if input_dataset_name in h5f and target_dataset_name in h5f:
                input_data = h5f[input_dataset_name][:].astype(np.float32)
                target_data = h5f[target_dataset_name][:].astype(np.float32)

                if combined_input is None:
                    combined_input = input_data
                    combined_target = target_data
                    # Free memory of the loaded data
                    del input_data, target_data
                    gc.collect()

                else:
                    print_memory_usage()
                    combined_input = np.vstack((combined_input, input_data))
                    combined_target = np.vstack((combined_target, target_data))
                    # Free memory of the loaded data
                    del input_data, target_data
                    gc.collect()

            else:
                print(f"Dataset {input_dataset_name} or {target_dataset_name} not found.")

        # Shuffling
        indices = np.arange(combined_input.shape[0])
        np.random.shuffle(indices)
        combined_input = combined_input[indices]
        combined_target = combined_target[indices]

        return combined_input, combined_target

# Example usage
X, y = load_combine_shuffle_data_optimized_hdf5()

Total memory: 376.23 GB
Available memory: 204.16 GB
Used memory: 162.32 GB
Memory usage percentage: 45.7%


In [4]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
def display(x_dataset, y_dataset, i):
    # Extract the i-th data point from both datasets
    x_data_point = x_dataset[i]
    y_data_point = y_dataset[i]

    # Check if the index is valid
    if x_data_point.shape != (20, 13, 21):
        raise ValueError("Invalid shape for the x data point")

    # Extracting the transverse momentum (pt) from the y dataset
    pt = y_data_point[8]  # Assuming the 9th variable is at index 8

    fig, ax_main = plt.subplots(figsize=(8, 6))
    divider = make_axes_locatable(ax_main)

    # Add row sum plot as an inset to the main plot
    ax_row = divider.append_axes("right", size="20%", pad=0.4)

    # Add column sum plot below the main plot
    ax_column = divider.append_axes("bottom", size="20%", pad=0.5)

    # Initial plot
    im = ax_main.imshow(x_data_point[0, :, :], cmap='plasma')
    ax_main.invert_yaxis()
    ax_main.set_yticks(np.arange(x_data_point.shape[1]))
    ax_main.set_xticks(np.arange(x_data_point.shape[2]))
    ax_main.set_xlim(0, x_data_point.shape[2]-1)
    ax_main.set_ylim(0, x_data_point.shape[1]-1)
    ax_main.grid(True, color='gray', alpha=0.7)

    # Function to update the animation
    def update(t):
        # Update main plot
        data = x_data_point[t, :, :]
        im.set_data(data)

        # Update row sum plot
        ax_row.clear()
        ax_row.barh(np.arange(data.shape[0]), np.sum(data, axis=1), color='red')
        ax_row.set_ylim(0, data.shape[0]-1)
        ax_row.set_yticks(np.arange(data.shape[0]))
        ax_row.set_xlim(np.min(x_data_point[:, :, :].sum(axis=2)) * 1.1, np.max(x_data_point[:, :, :].sum(axis=2)) * 1.1)
        ax_row.set_xlabel("Row Sum")

        # Update column sum plot
        ax_column.clear()
        ax_column.bar(np.arange(data.shape[1]), np.sum(data, axis=0), color='blue')
        ax_column.set_xlim(0, data.shape[1]-1)
        ax_column.set_xticks(np.arange(data.shape[1]))
        ax_column.set_ylim(np.min(x_data_point[:, :, :].sum(axis=1)) * 1.1, np.max(x_data_point[:, :, :].sum(axis=1)) * 1.1)
        ax_column.set_ylabel("Column Sum")

        # Update labels and grid
        ax_main.set_xlabel("X Position")
        ax_main.set_ylabel("Y Position")


        # Update title for the entire figure
        fig.suptitle(f"Timestep: {t+1} | Data Point: {i} | pt: {pt:.2f}")

    ani = FuncAnimation(fig, update, frames=8, repeat=True)
    plt.close()

    return HTML(ani.to_jshtml())

In [5]:
n = X.shape[0]
X = X.reshape(n,20,13,21)
display(X, y, 19)

In [6]:
def display_diff(x_dataset, y_dataset, i):
    # Extract the i-th data point from both datasets
    x_data_point = x_dataset[i]
    y_data_point = y_dataset[i]

    # Check if the index is valid
    if x_data_point.shape != (20, 13, 21):
        raise ValueError("Invalid shape for the x data point")

    # Extracting the transverse momentum (pt) from the y dataset
    pt = y_data_point[8]  # Assuming the 9th variable is at index 8

    fig, ax_main = plt.subplots(figsize=(8, 6))
    divider = make_axes_locatable(ax_main)

    # Add row sum plot as an inset to the main plot
    ax_row = divider.append_axes("right", size="20%", pad=0.4)

    # Add column sum plot below the main plot
    ax_column = divider.append_axes("bottom", size="20%", pad=0.5)

    # Initial plot
    im = ax_main.imshow(x_data_point[0, :, :], cmap='plasma')
    ax_main.invert_yaxis()
    ax_main.set_yticks(np.arange(x_data_point.shape[1]))
    ax_main.set_xticks(np.arange(x_data_point.shape[2]))
    ax_main.set_xlim(0, x_data_point.shape[2]-1)
    ax_main.set_ylim(0, x_data_point.shape[1]-1)
    ax_main.grid(True, color='gray', alpha=0.7)

    # Function to update the animation
    def update(t):
        # Calculate the difference between current and previous timestep
        if t == 0:
            data = x_data_point[t, :, :]
        else:
            data = x_data_point[t, :, :] - x_data_point[t-1, :, :]
        
        # Update main plot
        im.set_data(data)

        # Update row sum plot
        ax_row.clear()
        ax_row.barh(np.arange(data.shape[0]), np.sum(data, axis=1), color='red')
        ax_row.set_ylim(0, data.shape[0]-1)
        ax_row.set_yticks(np.arange(data.shape[0]))
        ax_row.set_xlim(np.min(data.sum(axis=1)) * 1.1, np.max(data.sum(axis=1)) * 1.1)
        ax_row.set_xlabel("Row Sum")

        # Update column sum plot
        ax_column.clear()
        ax_column.bar(np.arange(data.shape[1]), np.sum(data, axis=0), color='blue')
        ax_column.set_xlim(0, data.shape[1]-1)
        ax_column.set_xticks(np.arange(data.shape[1]))
        ax_column.set_ylim(np.min(data.sum(axis=0)) * 1.1, np.max(data.sum(axis=0)) * 1.1)
        ax_column.set_ylabel("Column Sum")

        # Update labels and grid
        ax_main.set_xlabel("X Position")
        ax_main.set_ylabel("Y Position")

        # Update title for the entire figure
        fig.suptitle(f"Timestep: {t+1} | Data Point: {i} | pt: {pt:.2f}")

    ani = FuncAnimation(fig, update, frames=20, repeat=True)
    plt.close()
    return HTML(ani.to_jshtml())

In [7]:
display_diff(X, y, 1299)

/tmp/ipykernel_1085173/1464127409.py:47: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax_row.set_xlim(np.min(data.sum(axis=1)) * 1.1, np.max(data.sum(axis=1)) * 1.1)
/tmp/ipykernel_1085173/1464127409.py:55: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax_column.set_ylim(np.min(data.sum(axis=0)) * 1.1, np.max(data.sum(axis=0)) * 1.1)


In [ ]:
def check_zero_differences_all(x_dataset, start):
    # Get the number of data points in the dataset
    num_data_points = x_dataset.shape[0]
    all_zero_diff = True
    # Iterate over all data points
    for i in range(num_data_points):
        # Extract the i-th data point from the dataset
        x_data_point = x_dataset[i]

        # Check if the index is valid
        if x_data_point.shape != (20, 13, 21):
            raise ValueError(f"Invalid shape for data point {i}")

        print(f"Checking data point {i}:")

        # Flag to track if all differences are zero
        
        # Iterate over time steps starting from the 9th time frame
        for t in range(start, 20):
            # Calculate the difference between current and previous timestep
            diff = x_data_point[t, :, :] - x_data_point[t-1, :, :]

            # Check if all differences are zero
            if not np.allclose(diff, 0):
                all_zero_diff = False
                print(f"  Timestep {t+1}: Non-zero differences exist.")
                break

    if all_zero_diff:
        print(f"  All differences are zero for timesteps {start} to 20.")

    print()  # Print an empty line for separation
check_zero_differences_all(X, 13)

In [8]:
X = X.reshape(-1, 20*13*21)
n = X.shape[0]
X = X.reshape(n,20,13,21)
X= X[:, :15, :, :]
print_memory_usage()
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
del X
del y
gc.collect()
print_memory_usage()
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
del X_temp
del y_temp
gc.collect()
print_memory_usage()

Total memory: 376.23 GB
Available memory: 204.83 GB
Used memory: 161.65 GB
Memory usage percentage: 45.6%


Total memory: 376.23 GB
Available memory: 207.63 GB
Used memory: 158.85 GB
Memory usage percentage: 44.8%
Total memory: 376.23 GB
Available memory: 207.61 GB
Used memory: 158.87 GB
Memory usage percentage: 44.8%


In [9]:
def to_sum(X, y):
    if X.shape[1:] == (15, 13, 21) and y.shape[1:] == (13, ):
        #y_expanded = np.repeat(y[:,7], 8, axis=0).reshape(-1, 8, 1)
        X_sum = np.sum(X, axis=3)
        X_sum.reshape(X_sum.shape[0],15,13,1)
        n = y.shape[0]
        # one_hot = np.zeros((n, 3))
        one_hot = np.zeros((n, 1))

        # Class 1: np.abs(y[:, 8]) > 2
        one_hot[np.abs(y[:, 8]) >= 2, 0] = 1
        # one_hot[np.abs(y[:, 8]) < 2, 1] = 1

        # # Class 2: np.abs(y[:, 8]) <= 2 and y[:, 13] == 1
        # one_hot[(np.abs(y[:, 8]) < 2) & (y[:, 8] > 0), 1] = 1

        # # Class 3: np.abs(y[:, 8]) <= 2 and y[:, 13] == -1
        # one_hot[(np.abs(y[:, 8]) < 2) & (y[:, 8] < 0), 2] = 1
        return X_sum, one_hot, y[:,7].reshape(-1, 1)
    else:
        raise ValueError("Wrong array shape!")

X_sum_train, y_sum_train, y0_train = to_sum(X_train, y_train)
X_sum_val, y_sum_val, y0_val = to_sum(X_val, y_val)
X_sum_test, y_sum_test, y0_test = to_sum(X_test, y_test)
del X_train
del y_train
del X_val
del y_val
del X_test
del y_test
gc.collect()
print_memory_usage()

Total memory: 376.23 GB
Available memory: 215.49 GB
Used memory: 150.99 GB
Memory usage percentage: 42.7%


In [63]:
np.any(np.isnan(X_sum_train), axis=(0, 1, 2))

array([False])

In [81]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import PReLU, Input, LSTM, Flatten, Concatenate, Dense, Conv2D, TimeDistributed, MaxPooling2D, ReLU, Dropout, BatchNormalization
from tensorflow.keras.layers import Lambda, Conv2DTranspose, Reshape
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy, KLDivergence
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

latent_dim = 7

encoder_inputs = keras.Input(shape=(15, 13, 1))
x = layers.Conv2D(4, 3, strides=1, padding="same")(encoder_inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv2D(8, 3, strides=1, padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Flatten()(x)
x = layers.Dense(16)(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()




Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_28 (InputLayer)          [(None, 15, 13, 1)]  0           []                               
                                                                                                  
 conv2d_14 (Conv2D)             (None, 15, 13, 4)    40          ['input_28[0][0]']               
                                                                                                  
 batch_normalization_15 (BatchN  (None, 15, 13, 4)   16          ['conv2d_14[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 re_lu_15 (ReLU)                (None, 15, 13, 4)    0           ['batch_normalization_15[0]

In [82]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(15 * 13 * 8)(latent_inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Reshape((15, 13, 8))(x)

x = layers.Conv2DTranspose(8, 3, strides=1, padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv2DTranspose(4, 3, strides=1, padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

decoder_outputs = layers.Conv2DTranspose(1, 3, activation='softplus', padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 7)]               0         
                                                                 
 dense_28 (Dense)            (None, 1560)              12480     
                                                                 
 batch_normalization_18 (Bat  (None, 1560)             6240      
 chNormalization)                                                
                                                                 
 re_lu_18 (ReLU)             (None, 1560)              0         
                                                                 
 reshape_20 (Reshape)        (None, 15, 13, 8)         0         
                                                                 
 conv2d_transpose_60 (Conv2D  (None, 15, 13, 8)        584       
 Transpose)                                                

In [83]:
def kl_divergence_loss(z_mean, z_log_var):
        kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
        return tf.reduce_mean(kl_loss)
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
        
            z_log_var = tf.clip_by_value(z_log_var, clip_value_min=-10.0, clip_value_max=10.0)
            reconstruction = self.decoder(z)
            reconstruction_loss = keras.losses.mean_squared_error(K.flatten(data), K.flatten(reconstruction))
            reconstruction_loss *= np.prod((15,13,1))
            epsilon = 1e-6
            z_log_var = z_log_var + epsilon
            kl_loss = kl_divergence_loss(z_mean, z_log_var)
            total_loss = reconstruction_loss + kl_loss
            
            tf.print("Reconstruction Loss:", reconstruction_loss)
            tf.print("KL Divergence Loss:", kl_loss)
            tf.print("Total Loss:", total_loss)
            tf.print("Z Mean:", z_mean)
            tf.print("Z Log Variance:", z_log_var)
            tf.print("Z:", z)
            tf.print("Reconstructed Output:", reconstruction)

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

vae = VAE(encoder, decoder)
vae.compile(optimizer='rmsprop')
X_sum_train = X_sum_train.reshape(X_sum_train.shape[0], 15, 13, 1)
vae.fit(X_sum_train, epochs=30, batch_size=1024)

Epoch 1/30
Reconstruction Loss: inf
KL Divergence Loss: 1377364.75
Total Loss: inf
Z Mean: [[-70.2124252 67.4388809 -32.6966896 ... 90.2431 -181.01062 -278.952515]
 [-696.418762 648.498474 -94.500473 ... 771.834229 -1064.13855 -1473.50659]
 [-87.6277313 -734.666504 -318.251953 ... 176.262604 -355.987488 -1458.03625]
 ...
 [33.3329773 -90.432785 -43.6844597 ... 126.786873 -56.6426163 -127.662498]
 [-902.6604 941.376892 -140.403763 ... 337.510345 -835.817688 -1119.5708]
 [-65.9095535 -370.360413 -163.515503 ... 713.078735 -128.618073 -467.064]]
Z Log Variance: [[-9.99999905 3.35397196 -9.99999905 ... 10.000001 10.000001 -9.99999905]
 [-9.99999905 -9.99999905 -9.99999905 ... 10.000001 10.000001 -9.99999905]
 [10.000001 -9.99999905 -9.99999905 ... -9.99999905 -9.99999905 -9.99999905]
 ...
 [-9.99999905 10.000001 -9.99999905 ... 10.000001 10.000001 -9.99999905]
 [-9.99999905 10.000001 -9.99999905 ... 10.000001 10.000001 10.000001]
 [10.000001 -9.99999905 -9.99999905 ... -9.99999905 10.00000

KeyboardInterrupt: 

In [1]:
trained_vae.save('/fs/ddn/sdf/group/atlas/d/hjia625/Smart_Pixel/VAE_best_unquantized_unpruned.h5')

NameError: name 'model' is not defined